<a href="https://colab.research.google.com/github/mlnoobs/mlops-project-study/blob/main/2022-06-13/homework/cnrywjd11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요 툴&라이브러리 설치

In [2]:
!pip install --upgrade pip
!pip install torch
!pip install torchvision
!pip install numpy
!pip install matplotlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## import

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
import torchvision
from matplotlib import pyplot as plt
import time


## ModelV3 클래스 정의


In [4]:
class ModelV3(torch.nn.Module):
    def __init__(self):
        super(ModelV3, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = torch.nn.Dropout(0.25)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.fc1 = torch.nn.Linear(9216, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## 인퍼런스 클래스 정의

In [8]:
class MnistInferencer:
    def __init__(self, model_path):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = torch.load(model_path).to(self.device)
        self.model.eval()

    def inference(self, image):
        with torch.no_grad():
          image = torch.from_numpy(image).float()
          image = image.to(self.device)
          output = self.model(image)

          #디멘젼을 유지 하지 않는다. 1차원으로 변경
          pred = output.argmax(dim=1, keepdim=False)

          return pred.cpu().numpy()[0]

## 인퍼런스 테스트

In [13]:
mn_infer = MnistInferencer('./model.pt')
inputs_data = np.load('./mnist_inputs.npy')

for input_data in inputs_data:
    testimg = np.expand_dims(input_data, axis=(0,1)) # (28,28) -> (1, 1, 28, 28) 튜플을 사용해서 차원 한꺼번에 추가
    result = mn_infer.inference(testimg).item()
    print(result)

0
1
2
3
4
5
6
7
8
9
